In [7]:
import os
from Pyro4 import Daemon, Proxy
import socket
import time

# Función para serializar posix.lstat en un diccionario

In [1]:
def diccionario_de_atributos(cosa):
    return {attr: getattr(cosa, attr) for attr in dir(cosa) if not attr.startswith('__')}

# Lanzar servidor de nombres

In [2]:
from utils import lanzar_backgounrd, lanzar_como_proceso
lanzar_backgounrd('pyro4-ns', a_menos_que='lsof -t -i TCP:9090 -sTCP:LISTEN')

Lanzando pyro4-ns con pid 2721


(True, 2721)

# Definir el objeto remoto a ser expuesto

In [ ]:
class Ejemplo(object):
    def lstat(self, path='.'):
        return os.lstat(path)

    def lstat_attr(self, path='.'):
        val = os.lstat(path)
        return diccionario_de_atributos(val)
    

# Definir la función que será utilizada como servidor

In [ ]:
def servidor():
    Daemon.serveSimple({
        Ejemplo(): 'ejemplo'
    }, ns=True, verbose=True, host=socket.gethostname())
    

## Con un [`context manager`](http://eigenhombre.com/2013/04/20/introduction-to-context-managers/) lanzamos el servidor en otro proceso y probamos

In [15]:
with lanzar_como_proceso(servidor) as s:
    time.sleep(.5)  # Esperar el establecimiento del servidor (registandose en el NS)
    objeto_remoto = Proxy('PYRONAME:ejemplo')
    print("-"*10)
    print(objeto_remoto.lstat_attr())
    print("-"*10)

Lanzando <function servidor at 0x1027a06e0> con el pid 3445.
----------Object <__main__.Ejemplo object at 0x1022b2b90>:
    uri = PYRO:obj_c6a405ccb37e4573a0c309d74633fd2f@greybird:49924
    name = ejemplo
Pyro daemon running.

{u'st_ctime': 1445293164.0, u'st_flags': 0, u'st_mtime': 1445293164.0, u'st_blocks': 0, u'st_nlink': 14, u'n_sequence_fields': 10, u'n_unnamed_fields': 3, u'st_gid': 20, u'st_gen': 0, u'st_rdev': 0, u'st_birthtime': 1437916096.0, u'st_ino': 29865223, u'st_dev': 16777221, u'st_uid': 501, u'st_size': 476, u'n_fields': 19, u'st_mode': 16877, u'st_blksize': 4096, u'st_atime': 1445293165.0}
----------


La salida de todos los procesos se entremezcla pero podemos observar que no ha fallado

In [17]:
with lanzar_como_proceso(servidor) as s:
    time.sleep(.5)  # Esperar el establecimiento del servidor (registandose en el NS)
    objeto_remoto = Proxy('PYRONAME:ejemplo')
    try:
        print(objeto_remoto.lstat())
    except Exception as e:
        print("Ocurrió un error ###%s###" % e)

Lanzando <function servidor at 0x1027a06e0> con el pid 3594.
Ocurrió un error ###don't know how to serialize class <type 'posix.stat_result'>. Give it vars() or an appropriate __getstate__###Object <__main__.Ejemplo object at 0x1022b2b90>:
    uri = PYRO:obj_47b5130d80fa4a0bb8e8d3924c790659@greybird:49954
    name = ejemplo
Pyro daemon running.

